In [1]:
import copy
import glob
import pickle
import warnings
from datetime import datetime, timedelta
from itertools import product
import joblib

import cartopy
import cartopy.crs as ccrs
import cartopy.feature
import cartopy.feature as cfeature
import cartopy.feature as cf
import cartopy.io.shapereader as shpreader
import matplotlib as mpl
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shapely.geometry as sgeom
import xarray as xr
from scipy import stats
from scipy.spatial.distance import cdist
from shapely import geometry
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# import sys
# sys.path.append("/glade/u/home/jhayron/WeatherRegimes/Scripts/")
# import cluster_analysis, narm_analysis, som_analysis

In [2]:
def get_cold_indx(ds, mo_init=9, mo_end=2):
    """
    Extract indices for cold season.
    Grabbing Sept thru February init, for Oct thru March predictions.
    """
    dt_array = pd.to_datetime(ds['time'])
    return xr.where((dt_array.month>=mo_init) | (dt_array.month<=mo_end), True, False)

# Read data

In [3]:
names_variables = ['z500','olr', 'sm_region', 'sst', 'st_region', 
                   'u10','sst_st','sst_st_region','sst_sm_region']

In [5]:
for name in names_variables:
    print(name)
    path_anomalies = '/glade/work/jhayron/Weather_Regimes/ERA5/Daily_1degree/anomalies/'
    if 'region' in name:
        ds_era_anomalies = xr.open_dataset(f'{path_anomalies}{name[:-7]}_anomalies_v1959_2010_detrended.nc')
    else:
        ds_era_anomalies = xr.open_dataset(f'{path_anomalies}{name}_anomalies_v1959_2010_detrended.nc')
    ds_era_anomalies = ds_era_anomalies.rolling(time=7,min_periods=4).mean().shift(time=-6)
    dow = pd.to_datetime(ds_era_anomalies.time).day_of_week
    # Select Mondays and Thursdays
    where_mon_thu = np.where((dow == 0)|(dow==3))[0][:-12]
    week1_anoms = ds_era_anomalies.isel(time=where_mon_thu)
    # Only cold months
    cold_indx = get_cold_indx(week1_anoms,10,3)
    week1_anoms = week1_anoms.isel(time=cold_indx)
    # Replace nans with zeros depending on the variable
    if name == 'sst':
        vals_temp = copy.deepcopy(week1_anoms['sst_anomalies'].values)
        where_land = np.load(f'{path_anomalies}where_land.npy')
        where_ocean = np.load(f'{path_anomalies}where_ocean.npy')
        for i in range(len(vals_temp)):
            vals_temp[i,where_land[0],where_land[1]] = 0
        week1_anoms = week1_anoms.assign(sst_anomalies = (week1_anoms.sst_anomalies.dims,vals_temp))
    
    if name == 'st_region':
        vals_temp = copy.deepcopy(week1_anoms[f'{name[:-7]}_anomalies'].values)
        where_land = np.load(f'{path_anomalies}where_land.npy')
        where_ocean = np.load(f'{path_anomalies}where_ocean.npy')
        for i in range(len(vals_temp)):
            vals_temp[i,where_ocean[0],where_ocean[1]] = 0
        week1_anoms = week1_anoms.assign(st_anomalies = (week1_anoms.st_anomalies.dims,vals_temp))
        
    if name == 'sm_region':
        vals_temp = copy.deepcopy(week1_anoms[f'{name[:-7]}_anomalies'].values)
        where_land = np.load(f'{path_anomalies}where_land.npy')
        where_ocean = np.load(f'{path_anomalies}where_ocean.npy')
        for i in range(len(vals_temp)):
            vals_temp[i,where_ocean[0],where_ocean[1]] = 0
        week1_anoms = week1_anoms.assign(sm_anomalies = (week1_anoms.sm_anomalies.dims,vals_temp))
        
    # Cut region depending on variable
    if 'region' in name:
        # region for clustering
        lat0=10; lat1=70; lon0=210; lon1=320
        week1_anoms = week1_anoms.where((week1_anoms.lon<=lon1)&(week1_anoms.lon>=lon0)\
                                        &(week1_anoms.lat<=lat1)&(week1_anoms.lat>=lat0),drop=True)
    else:
        week1_anoms = week1_anoms.where(week1_anoms.lat>-30,drop=True)
        
    week1_anoms.to_netcdf(f'/glade/work/jhayron/Weather_Regimes/weekly_anomalies/week1_{name}_anoms_v3.nc')
    
    del(week1_anoms)
    del(ds_era_anomalies)

    import gc
    gc.collect()

z500
olr
sm_region
sst
st_region
u10
sst_st
sst_st_region
sst_sm_region
